### Initialise

In [ ]:
## Import necessary packages
import time;
import numpy as np; 
import matplotlib.pyplot as plt; 
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import python.data as datameta

In [ ]:
# Basic Initialisations
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

### Preprocess Data

In [ ]:
data = datameta.processData('data/clean_data.csv'); 
data.head()

In [ ]:
# Split into x and y
x_keys = [
    "Time (h)", "Aeration rate(Fg:L/h)", "Sugar feed rate(Fs:L/h)","Acid flow rate(Fa:L/h)",
    "Base flow rate(Fb:L/h)","Heating/cooling water flow rate(Fc:L/h)","Heating water flow rate(Fh:L/h)",
    "Water for injection/dilution(Fw:L/h)","Substrate concentration(S:g/L)","PAA flow(Fpaa:PAA flow (L/h))",
    "Oil flow(Foil:L/hr)", "Oxygen Uptake Rate(OUR:(g min^{-1}))", "Ammonia shots(NH3_shots:kgs)",
    # Converted variables
    "0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)",
    "Air head pressure(pressure:bar)", "Temperature(T:K)", "pH(pH:pH)", "Vessel Volume(V:L)"
]
(x,y) = datameta.xy_split(data,x_keys)

In [ ]:
rand = round(time.time() * 1000) % 100; 
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=rand)

print ("x_train: ", x_train.shape)
print ("x_test: ", x_test.shape)

print ("y_train: ", y_train.shape)
print ("y_test: ", y_test.shape)

In [ ]:
# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.x.shape[0]
       
    def __getitem__(self, index):
        return self.x[index], self.y[index]
   
    def __len__(self):
        return self.len
   
batch_size = 32

# Instantiate training and test data
train_data = Data(x_train.to_numpy(), y_train.to_numpy())
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(x_test.to_numpy(), y_test.to_numpy())
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [ ]:
# Just checking if it worked
for batch, (x, y) in enumerate(train_dataloader):
    print(f"Batch: {batch+1}")
    print(f"X shape: {x.shape}")
    print(f"y shape: {y.shape}")
    break

In [ ]:
input_dim = len(x_keys)
hidden_dim = 950
output_dim = len(y_keys)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        self.layer_2 = nn.Linear(hidden_dim, output_dim)
       
    def forward(self, x):
        x = torch.nn.functional.relu(self.layer_1(x))
        x = self.layer_2(x)

        return x
       
model = NeuralNetwork(input_dim, hidden_dim, output_dim)
print(model)

In [ ]:
learning_rate = 0.00001
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.5)

In [ ]:
num_epochs = 1500
loss_values = []


for epoch in range(num_epochs):
    for x, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
       
        # forward + backward + optimize
        pred = model(x)
        loss = loss_fn(pred, y)
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
        # scheduler.step()

# plt.plot(loss_values[500:])
plt.plot(loss_values)
print("Training Complete")

In [ ]:
total = 0.0
correct = 0.0
y_pred = []

def isClose(base, known, tol):
    return np.abs((base - known) / base) <= tol

with torch.no_grad():
    for x, y in test_dataloader:
        outputs = model(x)
        np.append(y_pred, outputs)
        np.append(y_test, y)
        # y_pred.append(predicted)
        # y_test.append(y)
        total += y.size(0)
        correct += np.sum(isClose(outputs.numpy(), y.numpy(), tol=0.1))

print(f'Accuracy out of {total} test instances: {(100 * correct) / total}%')